In [1]:
import openai as client

assistant = client.beta.assistants.create(
    name="Book Assistant",
    instructions="You help users with their question on the files they upload.",
    model="gpt-4o-mini",
    tools=[{"type": "file_search"}],
)
assistant

# assistant_id = "asst_9xcCNAq3brRU5PhMxyPHT5O7"

Assistant(id='asst_W7SlRDQ4EuYW3NCwRWpI5eux', created_at=1726291479, description=None, instructions='You help users with their question on the files they upload.', metadata={}, model='gpt-4o-mini', name='Book Assistant', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=[])), top_p=1.0)

In [ ]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want you to help me with this file.",
        }
    ]
)
thread

In [ ]:
file = client.files.create(
    file = client.file_from_path(".cache/files/chapter_one.txt"),
    purpose="assistants",
)
file

In [ ]:
client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content="",
    attachments=[
        {
            'file_id':file.id,
            'tools':[
                {
                    "type":"file_search",
                }
            ]
        }
    ]
)

In [19]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)

In [30]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content,
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(
        thread_id=thread_id,
    )
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")
        for annotation in message.content[0].text.annotations:
            print(f"Source: {annotation.file_citation}")



def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        # print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outputs = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outputs,
    )

In [ ]:
get_run(run.id, thread.id).status

In [31]:
get_messages(thread.id)

user: I want you to help me with this file.
assistant: Sure! Please let me know what specific information or assistance you need regarding the file you've uploaded.
user: I want to know where does Winston live and how his apartment looks like.
assistant: Winston lives in a building called Victory Mansions. The description of his living situation and apartment includes several key details:

1. **Victory Mansions Description**: Winston enters through "the glass doors of Victory Mansions," which has a hallway that smells of boiled cabbage and old rag mats. The atmosphere is grimy and unwelcoming, suggesting a place in disrepair【6:0†source】.

2. **Apartment Details**: Winston's flat is located seven flights up, as the lift is usually not working. The entire environment around him is cold and bleak. Inside the flat, there is a telescreen that cannot be completely turned off, allowing the Party to monitor activities【6:0†source】【6:3†source】.

3. **Overall Atmosphere**: The bleakness is furthe

In [ ]:
submit_tool_outputs(run.id, thread.id)

In [ ]:
send_message(thread.id, "I want to know where does Winston live and how his apartment looks like.",)